In [1]:
import torch

%matplotlib inline

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)
print(device)

cuda


In [3]:
words = open("names.txt", "r").read().splitlines()

In [4]:
chars = sorted(list(set("".join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}

In [5]:
vocab_size = len(stoi)
word_count = len(words)
print(f"{vocab_size=}")
print(f"{word_count=}")

vocab_size=27
word_count=32033


In [6]:
block_size = 3


def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y


import random

random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

In [7]:
def cmp(s, dt: torch.Tensor, t: torch.Tensor):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(
        f"{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}"
    )

In [8]:
n_emb = 10
n_hidden = 64


g = torch.Generator(device=device).manual_seed(2147483647)
C = torch.randn((vocab_size, n_emb), generator=g)
w1 = (
    torch.randn((n_emb * block_size, n_hidden), generator=g)
    * (5 / 3)
    / ((n_emb * block_size) ** 0.5)
)
# Can remove bias due ot batchnorm removing its effect
b1 = torch.randn(n_hidden, generator=g) * 0.1
w2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

bn_gain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bn_bias = torch.randn((1, n_hidden)) * 0.1

params = [C, w1, b1, w2, b2, bn_gain, bn_bias]
print(sum(p.nelement() for p in params))
for p in params:
    p.requires_grad = True

4137


In [9]:
batch_size = 32
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [10]:
# Forward
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
hprebn = embcat @ w1

# -----------BatchNormalisation Layer ----------------
bn_meani = 1 / batch_size * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bn_meani
bndiff2 = bndiff**2
bnvar = (
    1 / (batch_size - 1) * (bndiff2).sum(0, keepdims=True)
)  # Divide by n-1 is Bessel's correction
bnvar_inv = (bnvar + 1e-5) ** -0.5
bn_raw = bndiff * bnvar_inv
hpreact = bn_gain * bn_raw + bn_bias

# ----------Non Linearity-----------------
h = torch.tanh(hpreact)

# ----------Linear Layer 2----------------
logits = h @ w2 + b2

# ----------Cross Entropy Loss------------
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
log_probs = probs.log()
loss = -log_probs[range(batch_size), Yb].mean()
print(loss)

# -----------PyTorch BackProp and Loss-----
# backward
for p in params:
    p.grad = None

for t in [
    log_probs,
    probs,
    counts,
    counts_sum,
    counts_sum_inv,
    norm_logits,
    logit_maxes,
    logits,
    h,
    hpreact,
    bn_raw,
    bnvar_inv,
    bnvar,
    bndiff2,
    bndiff,
    hprebn,
    bn_meani,
    embcat,
    emb,
]:
    t.retain_grad()

loss.backward()
loss
print(loss)

tensor(3.3725, device='cuda:0', grad_fn=<NegBackward0>)
tensor(3.3725, device='cuda:0', grad_fn=<NegBackward0>)
